In [ ]:
import angr
from angrutils import *

In [ ]:
# binary = '/data2/zhangzheng1/data/example-c/build/src/test_string'
# binary = './test'
binary = '/usr/bin/base64'

p = angr.Project(binary, auto_load_libs=False)

In [ ]:
# main = p.loader.main_object.get_symbol("main")
# print(hex(main.rebased_addr))
# print(hex(p.entry))

In [ ]:
# cfg = p.analyses.CFGEmulated(fail_fast=True, starts=[main.rebased_addr])
cfg = p.analyses.CFGFast()

In [26]:
cfg.kb.functions.get('main').get_call_sites()

dict_keys([4200256, 4200295, 4200310, 4200325, 4200335, 4200345, 4200542, 4200551, 4200645, 4200571, 4200484, 4200588, 4200524, 4200406, 4200430, 4200670, 4200877, 4200447, 4200892, 4200750, 4200688, 4200900, 4200707, 4200827, 4200783, 4200842, 4200800, 4200850, 4200808])

In [ ]:
# for b in sorted(cfg.kb.functions.get('main').blocks, key=lambda x: x.addr):
#     b.pp()
#     print()

In [ ]:
# [b for b in cfg.kb.functions.get('__libc_start_main').blocks][0].pp()

In [ ]:
# cfg.kb.functions.get('main')

In [ ]:
def checkNop(blk): # 检查blk全部为nop的情况
    try:
        if not blk: return True
        if blk.size == 0: return True
        return all([blk.capstone.insns[i].insn.insn_name() == 'nop' for i in range(blk.instructions)])
    except Exception as e:
        print('[NOP]{} {}'.format(blk, blk.size))
        return False

def getcov(func, s):

    blkList = list(func.blocks)

    for blk in blkList: # 遍历该函数所有block
        if blk.addr not in s:
            s.add(blk.addr)
            
            node = cfg.get_node(blk.addr) # 将block转换为node

            if node.has_return or checkNop(blk): # 该block包含ret则不遍历successors或全部为nop指令
                s.add(node.addr)
            else:
                for ns in node.successors: # 遍历该block的后续block
                    if ns.addr not in s:
                        if ns.has_return or checkNop(ns.block): # 同上
                            s.add(ns.addr)
                        else:
                            if ns.addr in func.block_addrs: # func内条件跳转（angr解析func得到的blk会少一部份）
                                continue
                            else:
                                try:
                                    new_func = cfg.kb.functions.get_by_addr(ns.addr) # jmp或call调用其它函数
                                    getcov(new_func, s) # 递归解析下一个函数
                                except KeyError as e: # func内跳转到angr未解析为该func的blk
                                    blkList.append(ns.block)
#                                     print('[!] {} {} {}'.format(func.name, hex(blk.addr), hex(ns.addr)))
                                    continue
    
    return len(s)

In [ ]:
print('[+] Functions: {}'.format(len(list(cfg.kb.functions.items()))))
print('[+] Nodes: {}'.format(len(cfg.graph.nodes())))

In [ ]:
from tqdm import tqdm

func2cov = {}

for addr, func in tqdm(cfg.kb.functions.items()):
    if func.name not in func2cov:
        func2cov[func.name] = getcov(func, set())
    else:
        func2cov[func.name] = max(func2cov[func.name], getcov(func, set()))

In [ ]:
main_cov = getcov(cfg.kb.functions.get('main'), set())
all_blk = len(cfg.graph.nodes())
print('[+] {:.2%}'.format(main_cov/all_blk))

In [ ]:
# sub_403162 0x40318f 0x403b9f
func = cfg.functions.get('sub_402e10')
# print(0x402e10 <= 0x403a10 < 0x40343f)
for b in sorted(func.blocks, key=lambda x: x.addr):
# for b in func.blocks:
#     if b.addr == 0x40336a:
    b.pp()
    print()
# print(hex(func.addr))
# print(hex(func.addr+func.size))
# print(func.size)

In [ ]:
# 0x403162 in func.block_addrs

In [22]:
cfg.get_node(0x403162).block.pp()

0x403162:	sub	rsp, 8
0x403166:	mov	r8d, r15d
0x403169:	mov	rcx, r13
0x40316c:	push	qword ptr [rsp + 0x58]
0x403170:	push	qword ptr [rsp + 0x68]
0x403174:	mov	rsi, r10
0x403177:	push	0
0x403179:	mov	r9d, dword ptr [rsp + 0x58]
0x40317e:	mov	rdi, r14
0x403181:	mov	rdx, qword ptr [rsp + 0x38]
0x403186:	and	r9d, 0xfffffffd
0x40318a:	call	0x402e10


In [23]:
cfg.get_node(0x403162).predecessors

[<CFGNode 0x402e10[109]>]

In [ ]:
main = cfg.functions.get('main')
blkcnt = 0
for blk in main.blocks:
    blkcnt += 1
    for s in cfg.get_node(blk.addr).successors:
        if not (main.addr <= s.addr <= main.addr + main.size):
            name = cfg.functions.get_by_addr(s.addr).name
            print('{} {}'.format(name, func2cov[name]))
print('main: {} {}'.format(blkcnt, func2cov['main']))

In [ ]:
for f in sorted(func2cov, key=lambda x: func2cov[x], reverse=True)[:40]:
    print('{}: {}'.format(f, func2cov[f]))

In [ ]:
plot_cfg(cfg, "test", asminst=True, remove_imports=True, remove_path_terminator=True)

In [27]:
main.block_addrs

dict_keys([4200256, 4200295, 4200310, 4200325, 4200335, 4200345, 4200368, 4200609, 4200377, 4200625, 4200542, 4200598, 4200386, 4200639, 4200635, 4200551, 4200388, 4200416, 4200734, 4200645, 4200571, 4200484, 4200395, 4200531, 4200421, 4200741, 4200658, 4200588, 4200524, 4200406, 4200430, 4200670, 4200877, 4200447, 4200683, 4200892, 4200476, 4200750, 4200688, 4200900, 4200768, 4200707, 4200922, 4200715, 4200770, 4200720, 4200776, 4200827, 4200783, 4200842, 4200800, 4200850, 4200808, 4200872, 4200825])